In [1]:
import os

In [2]:
os.chdir('../')

In [3]:
%pwd

'd:\\Manav\\Projects\\cnnClassifier'

In [4]:
import tensorflow as tf

In [5]:
model = tf.keras.models.load_model('artifacts/training/model.h5')

In [6]:
from dataclasses import dataclass
from pathlib import Path 

@dataclass(frozen=True)
class EvaluationConfig:
    model_path: Path
    training_data: Path
    all_params: dict
    params_image_size: list
    params_batch_size: int

In [7]:
from cnnClassifier.constants import *
from cnnClassifier.utils import *

In [21]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
    
    def get_validation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            model_path='artifacts/training/model.h5',
            training_data='artifacts/data_ingestion/PetImages',
            all_params= self.params,
            params_image_size= self.params.IMAGE_SIZE,
            params_batch_size= self.params.BATCH_SIZE
        )
        print(Path(os.path.join(self.config.data_ingestion.unzip_dir,'PetImages')))
        return eval_config

In [9]:
from urllib.parse import urlparse

In [12]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config
    
    def _valid_generator(self):
        
        datagenerator_kwargs = dict(
            rescale= 1./255,
            validation_split = 0.20
        )
        
        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            interpolation = 'bilinear'
        )
        
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )
        
        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory= self.config.training_data,
            subset='validation',
            shuffle= False,
            **dataflow_kwargs
        )
        
    
    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    
    def evaluation(self):
        self.model = self.load_model(self.config.model_path)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        
    
    def save_score(self):
        scores = {'Loss': self.score[0], 'Accuracy': self.score[1]}
        save_json(path=Path('scores.json'),data=scores)

In [22]:
try:
    config = ConfigurationManager()
    evaluation_config = config.get_validation_config()
    evaluation = Evaluation(config=evaluation_config)
    evaluation.evaluation()
    evaluation.save_score()
except Exception as e:
    raise e

[2024-04-01 17:00:08,195: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-04-01 17:00:08,197: INFO: common: yaml file: params.yaml loaded successfully]
[2024-04-01 17:00:08,199: INFO: common: created directory at: artifacts]
artifacts/data_ingestion\PetImages
Found 78 images belonging to 2 classes.
5/5 [==============================] - 0s 68ms/step - loss: 0.9929 - accuracy: 0.8718
[2024-04-01 17:00:09,144: INFO: common: json file saved at: scores.json]
